# Multithreading in Julia

## Spawning parallel tasks

In [1]:
using Base.Threads

@show nthreads();

nthreads() = 4


In [2]:
@time t = @spawn begin # `@spawn` returns right away
    sleep(2)
    3+3
end

@time fetch(t) # `fetch` waits for the task to finish

  0.008318 seconds (4.21 k allocations: 202.578 KiB, 99.24% compilation time)
  1.974720 seconds (219 allocations: 12.625 KiB, 0.07% compilation time)


6

## Example: multi-threaded `map`

In [3]:
using LinearAlgebra, BenchmarkTools

BLAS.set_num_threads(1) # Fix number of BLAS threads

function tmap(fn, itr)
    # for each i ∈ itr, spawn a task to compute fn(i)
    tasks = map(i -> @spawn(fn(i)), itr)
    # fetch and return all the results
    return fetch.(tasks)
end

M = [rand(200,200) for i in 1:8];

tmap(svdvals, M);

@btime  map(svdvals, $M) samples=10 evals=3;
@btime tmap(svdvals, $M) samples=10 evals=3;

  28.879 ms (106 allocations: 3.37 MiB)
  7.800 ms (155 allocations: 3.38 MiB)


***Exercise***: do you see any difference if you increase the number of BLAS threads?

## Example: multi-threaded `for` loop

In [4]:
using ChunkSplitters, Base.Threads, BenchmarkTools

function sum_threads(fn, data; nchunks=nthreads())
    psums = zeros(eltype(data), nchunks)
    @threads :dynamic for (c, elements) in enumerate(chunks(data; n=nchunks))
        psums[c] = sum(fn, elements)
    end
    return sum(psums)
end

v = randn(10_000_000);

@btime sum(sin, $v);

@btime sum_threads(sin, $v);

  86.676 ms (0 allocations: 0 bytes)
  23.455 ms (27 allocations: 2.38 KiB)


***Exercise***: do you see differences if you change the scheduler type?  Remember you can choose between `:dynamic` (currently the default if omitted), `:greedy`, and `:static`.

In [5]:
function sum_map_spawn(fn, data; nchunks=nthreads())
    ts = map(chunks(data, n=nchunks)) do elements
        @spawn sum(fn, elements)
    end
    return sum(fetch.(ts))
end

@btime sum_map_spawn(sin, $v);

  23.552 ms (48 allocations: 3.00 KiB)


In [6]:
using OhMyThreads: @tasks

function sum_tasks(fn, data; nchunks=nthreads())
    psums = zeros(eltype(data), nchunks)
    @tasks for (c, elements) in enumerate(chunks(data; n=nchunks))
        psums[c] = sum(fn, elements)
    end
    return sum(psums)
end

@btime sum_tasks(sin, $v);

  23.419 ms (32 allocations: 2.59 KiB)


## Multi-threading: is it always worth it?

In [7]:
using BenchmarkTools

function overhead!(v)
    for idx in eachindex(v)
        v[idx] = idx
    end
end
    
function overhead_threads!(v)
    @threads for idx in eachindex(v)
        v[idx] = idx
    end
end

N = 100

@btime overhead!(v) setup=(v = Vector{Int}(undef, N))
@btime overhead_threads!(v) setup=(v = Vector{Int}(undef, N))

  17.786 ns (0 allocations: 0 bytes)
  4.389 μs (22 allocations: 2.09 KiB)


***Exercise***: do you see any improvement in the parallel efficiency if you change the size of the problem (here: `N`)?

## Unbalanced workload: computing hexadecimal $\pi$

_This section is inspired by the blogpost [Computing the hexadecimal value of pi](https://giordano.github.io/blog/2017-11-21-hexadecimal-pi/) by Mosè Giordano._

The [Bailey–Borwein–Plouffe formula](https://en.wikipedia.org/wiki/Bailey%E2%80%93Borwein%E2%80%93Plouffe_formula) is one of the [several algorithms to compute $\pi$](https://en.wikipedia.org/wiki/Approximations_of_%CF%80):

$$
\pi = \sum_{k = 0}^{\infty}\left[ \frac{1}{16^k} \left( \frac{4}{8k + 1} -
\frac{2}{8k + 4} - \frac{1}{8k + 5} - \frac{1}{8k + 6} \right) \right]
$$

What makes this formula stand out among other approximations of $\pi$ is that it allows one to directly extract the $n$-th fractional digit of the hexadecimal value of $\pi$ without computing the preceding ones.

The Wikipedia article about the Bailey–Borwein–Plouffe formula explains that the $n + 1$-th fractional digit $d_n$ is given by

$$
d_{n} = 16 \left[ 4 \Sigma(n, 1) - 2 \Sigma(n, 4) - \Sigma(n, 5) - \Sigma(n,
6) \right]
$$

where

$$
\Sigma(n, j) = \sum_{k = 0}^{n} \frac{16^{n-k} \bmod (8k+j)}{8k+j} + \sum_{k
= n+1}^{\infty} \frac{16^{n-k}}{8k+j}
$$

Only the fractional part of expression in square brackets on the right side of $d_n$ is relevant, thus, in order to avoid rounding errors, when we compute each term of the finite sum above we can take only the fractional part. This allows us to always use ordinary double precision floating-point arithmetic, without resorting to arbitrary-precision numbers. In addition note that the terms of the infinite sum get quickly very small, so we can stop the summation when they become negligible.

### Serial implementation

In [8]:
# Return the fractional part of x, modulo 1, always positive
fpart(x) = mod(x, one(x))

function Σ(n, j)
    # Compute the finite sum
    s = 0.0
    denom = j
    for k in 0:n
        s = fpart(s + powermod(16, n - k, denom) / denom)
        denom += 8
    end
    # Compute the infinite sum
    num = 1 / 16
    while (frac = num / denom) > eps(s)
        s     += frac
        num   /= 16
        denom += 8
    end
    return fpart(s)
end

pi_digit(n) =
    floor(Int, 16 * fpart(4Σ(n-1, 1) - 2Σ(n-1, 4) - Σ(n-1, 5) - Σ(n-1, 6)))

pi_string(n) = "0x3." * join(string.(pi_digit.(1:n), base = 16)) * "p0"

pi_string (generic function with 1 method)

Let's make sure this works:

In [9]:
pi_string(13)

"0x3.243f6a8885a30p0"

In [10]:
# Parse the string as a double-precision floating point number...
parse(Float64, pi_string(13))

3.141592653589793

In [11]:
# ...or get the double-precision value by summing all the hexadecimal digits
3 + sum(pi_digit(n)/16^n for n in 1:13)

3.141592653589793

In [12]:
Float64(π) == parse(Float64, pi_string(13))

true

In [13]:
precision(BigFloat)

256

In [14]:
pi_string(64)

"0x3.243f6a8885a308d313198a2e03707344a4093822299f31d0082efa98ec4e6c89p0"

In [15]:
3 + sum(pi_digit(n)/big(16)^n for n in 1:64)

3.141592653589793238462643383279502884197169399375105820974944592307816406286198

In [16]:
BigFloat(π) == parse(BigFloat, pi_string(64))

true

In [17]:
setprecision(BigFloat, 4000) do
    BigFloat(π) == parse(BigFloat, pi_string(1000))
end

true

In [18]:
using BenchmarkTools

b = @benchmark pi_string(1000)

pi_serial_t = minimum(b.times)

b

BenchmarkTools.Trial: 13 samples with 1 evaluation.
 Range (min … max):  389.169 ms … 398.948 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     395.630 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   395.020 ms ±   2.627 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                          █                      
  ▆▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▆▁▆█▁▁▆▆▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  389 ms           Histogram: frequency by time          399 ms <

 Memory estimate: 66.89 KiB, allocs estimate: 2015.

### Multi-threaded implementation

Since the Bailey–Borwtimesn–Plouffe formula extracts the $n$-th digit of $\pi$ without computing the other ones, we can write a multi-threaded version of `pi_string`, taking advantage of native support for [multi-threading](https://docs.julialang.org/en/v1/manual/multi-threading/) in Julia. However note that the computational cost of `pi_digit` is $O(nlog(n))$, so the larger the value of $n$, the longer the function will take, which makes this workload very unbalanced. ***Question***: what do you expect to be the worst performing scheduler?

#### For-loop: static scheduler

In [19]:
function pi_string_threads_static(N)
    digits = Vector{Int}(undef, N)
    @threads :static for n in eachindex(digits)
        digits[n] = pi_digit(n)
    end
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

pi_string_threads_static(1000) == pi_string(1000)

true

In [20]:
b = @benchmark pi_string_threads_static(1000)

pi_threads_static_t = minimum(b.times)

b

BenchmarkTools.Trial: 27 samples with 1 evaluation.
 Range (min … max):  188.677 ms … 196.517 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     190.008 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   190.615 ms ±   2.066 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ ▃  █    ▃▃        ▃                                          
  █▇█▁▇█▁▇▇▇██▇▁▁▁▇▁▁▁█▁▁▁▇▁▁▁▁▁▇▇▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▇ ▁
  189 ms           Histogram: frequency by time          197 ms <

 Memory estimate: 76.95 KiB, allocs estimate: 2042.

In [21]:
pi_serial_t / pi_threads_static_t / nthreads() * 100

51.565658230361144

#### For-loop: dynamic scheduler

In [22]:
function pi_string_threads_dynamic(N)
    digits = Vector{Int}(undef, N)
    @threads :dynamic for n in eachindex(digits)
        digits[n] = pi_digit(n)
    end
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

pi_string_threads_dynamic(1000) == pi_string(1000)

true

In [23]:
b = @benchmark pi_string_threads_dynamic(1000)

pi_threads_dynamic_t = minimum(b.times)

b

BenchmarkTools.Trial: 26 samples with 1 evaluation.
 Range (min … max):  188.327 ms … 233.806 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     190.173 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   192.505 ms ±   8.766 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ██ ▂                                                          
  ▆██▆█▄▁▁▄▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  188 ms           Histogram: frequency by time          234 ms <

 Memory estimate: 76.95 KiB, allocs estimate: 2042.

In [24]:
pi_serial_t / pi_threads_dynamic_t / nthreads() * 100

51.66151437580111

#### For-loop: greedy scheduler

In [25]:
function pi_string_threads_greedy(N)
    digits = Vector{Int}(undef, N)
    @threads :greedy for n in eachindex(digits)
        digits[n] = pi_digit(n)
    end
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

pi_string_threads_greedy(1000) == pi_string(1000)

true

In [26]:
b = @benchmark pi_string_threads_greedy(1000)

pi_threads_greedy_t = minimum(b.times)

b

BenchmarkTools.Trial: 44 samples with 1 evaluation.
 Range (min … max):  110.462 ms … 133.489 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     112.305 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   114.226 ms ±   4.670 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██                                                             
  ██▄▆▇▄▄▇▄▄▄▄▄▁▄▁▄▁▄▄▁▁▆▄▁▆▄▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  110 ms           Histogram: frequency by time          133 ms <

 Memory estimate: 86.50 KiB, allocs estimate: 2561.

In [27]:
pi_serial_t / pi_threads_greedy_t / nthreads() * 100

88.0777450483477

#### Tasks

In [28]:
function pi_string_tasks(N)
    tasks = [Threads.@spawn pi_digit(n) for n in 1:N]
    digits = [fetch(t) for t in tasks]
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

pi_string_tasks(1000) == pi_string(1000)

true

In [29]:
b = @benchmark pi_string_tasks(1000)

pi_tasks_t = minimum(b.times)

b

BenchmarkTools.Trial: 48 samples with 1 evaluation.
 Range (min … max):  104.091 ms … 114.078 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     105.149 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   106.160 ms ±   2.415 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                              
  ██▅█▆▃▁▅▁▃▁▁▃▃█▃▃▅▃▃▁▁▁▃▃▁▁▃▁▁▁▁▁▁▃▁▁▃▃▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▃ ▁
  104 ms           Histogram: frequency by time          114 ms <

 Memory estimate: 567.61 KiB, allocs estimate: 7036.

In [30]:
pi_serial_t / pi_tasks_t / nthreads() * 100

93.46867797009104